# 1.0 Data Wrangling

## Introduction <a name="introduction"></a>
- #### This notebook explores the raw data given by Census.gov and Kaggle api's as well as the data downloaded from the Colorado Department of Education. 
- #### It will guide feature extraction from the datasets to a more human readable format.
- #### Finally, it will reduce the number of datasets to a more manageable amount to aid in exploratory data analysis. 

## Table of contents
* [Introduction](#introduction)
* [1 Getting Data](#get-data)
* [2 Making Datasets](#make-datasets)
    * [2.1 Census](#make-census)
    * [2.2 Expenditures](#make-expenditures)
    * [2.3 Kaggle](#make-kaggle)
        * [2.3.1 1 & 3 Year Change](#make-change)
        * [2.3.2 COACT](#make-coact)
        * [2.3.3 Enroll Working](#make-enrl)
        * [2.3.4 Final Grade](#make-final)
        * [2.3.5 k-12 Free or Reduced Lunch](#make-frl)
        * [2.3.6 Remediation High School](#make-remediation)
        * [2.3.7 School Adddress](#make-address)
        * [2.3.8 School GPS](#make-gps)
* [Conclusion](#conclusion)

In [2]:
import numpy as np
import pandas as pd

# To modules necessary to load the src.data.get_raw_data
import importlib
import sys

# setting path
sys.path.append('..')
# importing
from src.data import get_raw_data, make_datasets
importlib.reload(get_raw_data);
importlib.reload(make_datasets);

## 1. Getting Data <a id="get-data"></a>
Data was obtained from the following sources
    - [SAIPE Datasets (census.gov)](https://www.census.gov/programs-surveys/saipe/data/datasets.html): The number of students whose families below the poverty level by district for the years 2010-2012 through an API.
    - [School District Revenues and Expenditures | CDE (state.co.us)](http://www.cde.state.co.us/cdefinance/revexp): Revenue and expenditures by program and district for the years 2010-2012 which is downloaded manually.
    - [Visualize the State of Public Education in Colorado | Kaggle](https://www.kaggle.com/competitions/visualize-the-state-of-education-in-colorado/data?select=2010_1YR_3YR_change.csv): Student performance, college-readiness, and demographic information by school for the years 2010-2012 through an API

In [134]:
# Load the raw data 
raw_filepath = '../data/raw/' # filepath to all raw data
get_raw_data.get_census(raw_filepath + 'census/') # loads SAIPE data sets from census.gov saved as saipe_'year'
get_raw_data.get_kaggle(raw_filepath + 'kaggle/') # obtains the Kaggle competition data with their original names

## 2 Making Datasets <a id="make-datasets"></a>
The output produced by these api's and the files downloaded are in an extremely poor format especially the expenditures files. They need to be revised and improved in order to be used for exploratory data analysis. Ultimately, we will create a module named src.make_datasets to complete this process, so that it can be scaled. We will use this notebook to investigate how to do this effectively. Furthermore, we will try to define uniform column names such as district_name, school, or county to be used later when we join them. Finally, these datasets will be saved in a tall format in the interim folder. In other words, we will concatenate each year of the same datasets to with an additional year column to distinguish them.

In [143]:
# The filepath to save usable datasets to
interim_filepath = '../data/interim/'

### 2.1 Census <a id='make-census'></a>

In [130]:
census = pd.read_csv('../data/raw/census/saipe2010.csv', index_col=0)
census.head()

,SD_NAME,SAEPOV5_17RV_PT,SAEPOV5_17V_PT,SAEPOVALL_PT,time,state,school district (unified)
0,Cheyenne County School District RE-5,39,229,1350,2010,8,1
1,Yuma School District 1,179,977,5017,2010,8,16
2,Wray School District RD-2,103,727,3958,2010,8,17
3,Idalia School District RJ-3,12,92,502,2010,8,18
4,Liberty School District J-4,7,69,507,2010,8,19


__The census data is in pretty good shape, but there are some changes that can be made__
* The census columns should be renamed to be more intuitive. We will use the following map, 
    - SD_NAME : district_name
    - SAEPOV5_17RV_PT : est_child_poverty
    - SAEPOV5_17V_PT: est_total_child
    - SAEPOVALL_PT : est_total_pop
    - time : year
* We will also drop a few columns that we don't need
    - state
    - school district (unified)

In [138]:
make_datasets.make_tall_census(raw_filepath + 'census/', interim_filepath)

In [142]:
pd.read_csv(interim_filepath + 'saipe_tall.csv', index_col=0)

,district_name,est_child_poverty,est_total_child,est_total_pop,year
0,Cheyenne County School District RE-5,39,229,1350,2010
1,Yuma School District 1,179,977,5017,2010
2,Wray School District RD-2,103,727,3958,2010
3,Idalia School District RJ-3,12,92,502,2010
4,Liberty School District J-4,7,69,507,2010
...,...,...,...,...,...
173,Wiggins School District RE-50J,80,518,2556,2012
174,Wiley School District RE-13-JT,33,192,937,2012
175,Windsor School District RE-4,362,4722,22434,2012
176,Woodland Park School District RE-2,416,3042,18577,2012


### 2.2 Expenditures <a id="make-expenditures"><a/>

In [23]:
expenditures = pd.read_csv('../data/raw/expenditures/expenditures2010.csv')
expenditures

,Unnamed: 0,Unnamed: 1,DISTRICT/,Unnamed: 3,Total,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,BOCES,Total,Support,Community,Other,Total
1,NaN,COUNTY,FPC,Instruction,Services,Services,Expenditures,Expenditures
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
4,$,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
...,...,...,...,...,...,...,...,...
1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,NaN,STATE TOTALS,NaN,NaN,NaN,NaN,NaN,NaN
1014,$,Amount,NaN,"4,422,714,161","3,067,303,089","42,807,954","2,194,922,404","9,727,747,609"
1015,$,Per Pupil,"798,599.5","5,538","3,841",54,"2,748","12,181"


__The expenditures datasets are incredibly messy and will need a lot of fixing up. The end result that we want to obtain is a dataframe with the following columns__
* district_name
* county
* and the amount and per pupil amount for each category of spending

Also, we will be able to drop all BOCES funding as this not applicable to our project

In [24]:
# We can drop two things
# the empty rows between each district's information
# and the Unnamed: 0 column which just explains the units for each row.
expenditures = expenditures.dropna(how='all').drop('Unnamed: 0', axis=1)
expenditures.head()

,Unnamed: 1,DISTRICT/,Unnamed: 3,Total,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,BOCES,Total,Support,Community,Other,Total
1,COUNTY,FPC,Instruction,Services,Services,Expenditures,Expenditures
3,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
4,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
5,Per Pupil,"7,193.3","5,556","3,303",33,761,"9,653"


In [25]:
# Lets rename the columns that explain things better
expenditures.columns = ['county', 'district_name', 'instruction', 'support', 'community', 'other', 'sum']
expenditures.head()

,county,district_name,instruction,support,community,other,sum
0,NaN,BOCES,Total,Support,Community,Other,Total
1,COUNTY,FPC,Instruction,Services,Services,Expenditures,Expenditures
3,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
4,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
5,Per Pupil,"7,193.3","5,556","3,303",33,761,"9,653"


In [26]:
# Now we can drop the first two rows which contained column name information
expenditures = expenditures.drop([0,1])
expenditures = expenditures.reset_index(drop=True)
expenditures

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
2,Per Pupil,"7,193.3","5,556","3,303",33,761,"9,653"
3,All Funds,NaN,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
785,All Funds,NaN,50.4,47.7,0.6,1.3,100.0
786,STATE TOTALS,NaN,NaN,NaN,NaN,NaN,NaN
787,Amount,NaN,"4,422,714,161","3,067,303,089","42,807,954","2,194,922,404","9,727,747,609"
788,Per Pupil,"798,599.5","5,538","3,841",54,"2,748","12,181"


In [27]:
# All numbers have commas in them that need to be removed
expenditures = expenditures.replace(',','', regex=True)
expenditures

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,NaN,39962942,23760636,237704,5474579,69435862
2,Per Pupil,7193.3,5556,3303,33,761,9653
3,All Funds,NaN,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
785,All Funds,NaN,50.4,47.7,0.6,1.3,100.0
786,STATE TOTALS,NaN,NaN,NaN,NaN,NaN,NaN
787,Amount,NaN,4422714161,3067303089,42807954,2194922404,9727747609
788,Per Pupil,798599.5,5538,3841,54,2748,12181


In [28]:
# The district_name column has numbers that were relevant to the BOCES funding but not our project.
# We want to be able to identify each of those and remove them.
def remove_floats(entry):
    try:
        float(entry)
        return np.nan
    except:
        return entry

In [29]:
expenditures['district_name'] = expenditures['district_name'].apply(remove_floats)
expenditures.head()

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,NaN,39962942,23760636,237704,5474579,69435862
2,Per Pupil,NaN,5556,3303,33,761,9653
3,All Funds,NaN,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN


In [30]:
# Now that they are removed, lets forward fill the district_name,
# so that we can extract the total amount for each category
# and the per pupil amount for each category
expenditures['district_name'] = expenditures['district_name'].fillna(method='ffill')
expenditures

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,MAPLETON 1,39962942,23760636,237704,5474579,69435862
2,Per Pupil,MAPLETON 1,5556,3303,33,761,9653
3,All Funds,MAPLETON 1,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
785,All Funds,UTE PASS BOCES,50.4,47.7,0.6,1.3,100.0
786,STATE TOTALS,UTE PASS BOCES,NaN,NaN,NaN,NaN,NaN
787,Amount,UTE PASS BOCES,4422714161,3067303089,42807954,2194922404,9727747609
788,Per Pupil,UTE PASS BOCES,5538,3841,54,2748,12181


In [31]:
# We noticed that there are BOCES funding entries at the bottom of the project.
# We will remove these, because they are not relevant to the project
expenditures = expenditures[~(expenditures['district_name'].str.lower().str.contains('boces'))]

In [32]:
# Now we can extract the total amounts
totals = expenditures[expenditures['county'].str.lower() == 'amount'].drop('county', axis=1).reset_index(drop=True)
totals

,district_name,instruction,support,community,other,sum
0,MAPLETON 1,39962942,23760636,237704,5474579,69435862
1,ADAMS 12 FIVE STAR,220263102,130356958,943009,83867989,435431057
2,ADAMS COUNTY 14,34792431,34598159,703485,7862015,77956091
3,BRIGHTON 27J,62557927,50921503,219987,18440305,132139723
4,BENNETT 29J,4853879,3540284,0,1004359,9398521
...,...,...,...,...,...,...
174,YUMA 1,4431710,3277849,0,1358007,9067566
175,WRAY RD-2,3532489,2746335,0,800124,7078947
176,IDALIA RJ-3,1154510,810850,0,25002,1990362
177,LIBERTY J-4,926415,647462,0,25386,1599262


In [33]:
# the per pupil amounts 
per_pupils = expenditures[expenditures['county'].str.lower() == 'per pupil'].drop('county', axis=1).reset_index(drop=True)
per_pupils

,district_name,instruction,support,community,other,sum
0,MAPLETON 1,5556,3303,33,761,9653
1,ADAMS 12 FIVE STAR,5482,3244,23,2087,10837
2,ADAMS COUNTY 14,4956,4928,100,1120,11104
3,BRIGHTON 27J,4397,3579,15,1296,9287
4,BENNETT 29J,4537,3309,0,939,8784
...,...,...,...,...,...,...
174,YUMA 1,5596,4139,0,1715,11450
175,WRAY RD-2,5378,4181,0,1218,10776
176,IDALIA RJ-3,8539,5997,0,185,14722
177,LIBERTY J-4,11068,7736,0,303,19107


In [151]:
# and the county names
counties = expenditures.loc[~(expenditures['county'].str.lower().isin(('amount', 'per pupil', 'all funds'))), ['district_name', 'county']].reset_index(drop=True)
counties.head()

,district_name,county
0,MAPLETON 1,ADAMS
1,ADAMS 12 FIVE STAR,ADAMS
2,ADAMS COUNTY 14,ADAMS
3,BRIGHTON 27J,ADAMS
4,BENNETT 29J,ADAMS


In [121]:
# Before joining these dataframes, we will first make sure that their shapes are the same
print(f'{totals.shape=}',
      f'{per_pupils.shape=}',
     f'{counties.shape=}')

totals.shape=(179, 6) per_pupils.shape=(179, 6)


In [152]:
# Now we can merge them
df = pd.merge(left=totals, right=per_pupils, on='district_name', suffixes=('_total', '_per_pupil'))
pd.merge(left=df, right=counties, on='district_name')

,district_name,instruction_total,support_total,community_total,other_total,sum_total,instruction_per_pupil,support_per_pupil,community_per_pupil,other_per_pupil,sum_per_pupil,county
0,MAPLETON 1,39962942.37,23760636.02,237704.25,5474578.91,69435861.55,5556,3303,33,761,9653,ADAMS
1,ADAMS 12 FIVE STAR,220263101.67,130356957.5,943009.14,83867988.86,435431057.17,5482,3244,23,2087,10837,ADAMS
2,ADAMS COUNTY 14,34792431.43,34598159.34,703485.37,7862015.31,77956091.45,4956,4928,100,1120,11104,ADAMS
3,BRIGHTON 27J,62557927.35,50921503.01,219986.8,18440305.37,132139722.53,4397,3579,15,1296,9287,ADAMS
4,BENNETT 29J,4853878.63,3540283.5,0,1004358.98,9398521.11,4537,3309,0,939,8784,ADAMS
...,...,...,...,...,...,...,...,...,...,...,...,...
174,YUMA 1,4431710.03,3277848.7,0,1358006.84,9067565.57,5596,4139,0,1715,11450,YUMA
175,WRAY RD-2,3532489.32,2746334.57,0,800123.51,7078947.4,5378,4181,0,1218,10776,YUMA
176,IDALIA RJ-3,1154509.66,810850.16,0,25001.77,1990361.59,8539,5997,0,185,14722,YUMA
177,LIBERTY J-4,926414.66,647461.83,0,25385.95,1599262.44,11068,7736,0,303,19107,YUMA


In [257]:
make_datasets.make_tall_expenditures(raw_filepath + 'expenditures/', interim_filepath)

In [258]:
pd.read_csv(interim_filepath + 'expenditures_tall.csv', index_col=0)

,district_name,instruction_total,support_total,community_total,other_total,sum_total,instruction_per_pupil,support_per_pupil,community_per_pupil,other_per_pupil,sum_per_pupil,county,year
0,MAPLETON 1,39962942,23760636,237704,5474579,69435862,5556.0,3303.0,33,761.0,9653.0,ADAMS,2010
1,ADAMS 12 FIVE STAR,220263102,130356958,943009,83867989,435431057,5482.0,3244.0,23,2087.0,10837.0,ADAMS,2010
2,ADAMS COUNTY 14,34792431,34598159,703485,7862015,77956091,4956.0,4928.0,100,1120.0,11104.0,ADAMS,2010
3,BRIGHTON 27J,62557927,50921503,219987,18440305,132139723,4397.0,3579.0,15,1296.0,9287.0,ADAMS,2010
4,BENNETT 29J,4853879,3540284,0,1004359,9398521,4537.0,3309.0,0,939.0,8784.0,ADAMS,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,WRAY RD-2,3933428,2857350,0,6011673,12802451,5749.0,4176.0,0,8786.0,18712.0,YUMA,2012
176,IDALIA RJ-3,1058571,957184,0,288434,2304189,6786.0,6136.0,0,1849.0,14770.0,YUMA,2012
177,LIBERTY J-4,853026,707357,0,0,1560383,10978.0,9104.0,0,0.0,20082.0,YUMA,2012
178,CHARTER SCHOOL INSTITUTE,40561158,40549832,254978,6088775,87454743,3593.0,3592.0,23,539.0,7746.0,NaN,2012


### 2.3 Kaggle <a id="make-kaggle"></a>
There are many different Kaggle datasets for each year that were obtained using the API. Not all of them will be useful to our project. We will consider each of them and determine changes that need to be made to make our data analysis smooth. The information we obtain here will be implement in make_datasets.make_kaggle for every useful dataframe.
1. [1YR_3YR_change](#make--change): Provides the trend arrows for the school
2. [COACT](#make-coact): Specifies whether or not students are ready for college based on their ACT scores 
3. [enrl_working](#make-enrl): Contains demographics on the student population
4. [final_grade](#make-final): __This is the most important worksheet containing the achievement grades for each school__ 
5. [k_12_FRL](#make-frl): Shows the whether or not students qualify for free or reduced price lunches
6. [remediation_HS](#make-remediation): Identifies whether or not students graduating require remediation
7. [school_address](#make-address): Shows the schools' locations
8. [school_gps_coordinates](#make-gps): Shows the schools' GPS coordinates

__We will not use this dataset or file__
* data_map: Gives a dictionary of meaning of each feature across all datasets 
* DetailedGradingLogic : A pdf describing detailed grading logic

__To remain consistent across all dataframes, we will change the following columns accordingly__

In [22]:
kaggle_map = {'School Name': 'school',
              'District Number': 'district_id',
              'District Name': 'district_name',
              'School Number': 'school_id'}

__We will also be__ 
* Removing all rows that are completely empty
* making all column names lowercase and adding underscores between their words
* Removing any BOCES districts

#### 2.3.1 1 & 3 Year Change <a id="make-1-3-year-change"></a>

In [10]:
kaggle_change = pd.read_csv('../data/raw/kaggle/2010_1YR_3YR_change.csv')
kaggle_change

,School Name,District Number,District Name,EMH,EMH_combined,School Number,rate_at.5_chng_ach,rate_at.5_chng_growth,pct_pts_chnge_.5
0,ABRAHAM LINCOLN HIGH SCHOOL,880.0,DENVER COUNTY 1,H,NaN,10.0,3.0,2.0,1.0
1,ACADEMY CHARTER SCHOOL,900.0,DOUGLAS COUNTY RE 1,E,NaN,11.0,3.0,2.0,2.0
2,ACADEMY CHARTER SCHOOL,900.0,DOUGLAS COUNTY RE 1,M,NaN,11.0,3.0,3.0,3.0
3,ACRES GREEN ELEMENTARY SCHOOL,900.0,DOUGLAS COUNTY RE 1,E,NaN,12.0,2.0,2.0,2.0
4,GLACIER PEAK ELEMENTARY SCHOOL,20.0,ADAMS 12 FIVE STAR SCHOOLS,E,NaN,14.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...
2083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Before discussing all changes, it looks like EMH and EMH_combined might contained redundant information lets investigate this

In [4]:
kaggle_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2088 entries, 0 to 2087
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   School Name            2082 non-null   object 
 1   District Number        2082 non-null   float64
 2   District Name          2082 non-null   object 
 3   EMH                    2022 non-null   object 
 4   EMH_combined           254 non-null    object 
 5   School Number          2082 non-null   float64
 6   rate_at.5_chng_ach     1821 non-null   float64
 7   rate_at.5_chng_growth  1788 non-null   float64
 8   pct_pts_chnge_.5       1892 non-null   float64
dtypes: float64(5), object(4)
memory usage: 146.9+ KB


In [5]:
kaggle_change['EMH'].value_counts()

E    1043
M     533
H     446
Name: EMH, dtype: int64

In [6]:
kaggle_change['EMH_combined'].value_counts()

EM     140
MH      60
EMH     54
Name: EMH_combined, dtype: int64

In [5]:
# Check if there are any times were EMH_combined exists and EMH doesn't exist
kaggle_change[(kaggle_change['EMH'].isna()) & kaggle_change['EMH_combined'].notna()]

,School Name,District Number,District Name,EMH,EMH_combined,School Number,rate_at.5_chng_ach,rate_at.5_chng_growth,pct_pts_chnge_.5


In [7]:
kaggle_change[(kaggle_change['EMH'].isna()) & kaggle_change['EMH_combined'].isna()]

,School Name,District Number,District Name,EMH,EMH_combined,School Number,rate_at.5_chng_ach,rate_at.5_chng_growth,pct_pts_chnge_.5
60,ARVADA K-8,1420.0,JEFFERSON COUNTY R-1,NaN,NaN,109.0,NaN,NaN,NaN
68,ALICE TERRY ELEMENTARY SCHOOL,123.0,SHERIDAN 2,NaN,NaN,138.0,NaN,NaN,NaN
91,APS ONLINE SCHOOL,180.0,ADAMS-ARAPAHOE 28J,NaN,NaN,219.0,NaN,NaN,NaN
169,AYRES ELEMENTARY SCHOOL,1828.0,VALLEY RE-1,NaN,NaN,515.0,NaN,NaN,NaN
170,ACADEMY FOR ADVANCED AND CREATIVE LEARNING,1010.0,COLORADO SPRINGS 11,NaN,NaN,517.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
len(kaggle_change['EMH'].isna()) - len(kaggle_change['EMH_combined'].isna())

0

In [8]:
kaggle_change.iloc[30:32]

,School Name,District Number,District Name,EMH,EMH_combined,School Number,rate_at.5_chng_ach,rate_at.5_chng_growth,pct_pts_chnge_.5
30,AGATE JUNIOR-SENIOR HIGH SCHOOL,960.0,AGATE 300,H,MH,48.0,NaN,NaN,3.0
31,AGATE JUNIOR-SENIOR HIGH SCHOOL,960.0,AGATE 300,M,MH,48.0,NaN,NaN,3.0


It appears there are some entries where neither emh column is given

In [16]:
# Create dataframes for where we know EMH and EMH_combined exists
both_values = kaggle_change[~(kaggle_change['EMH'].isna()) & ~(kaggle_change['EMH_combined'].isna())]
# And where we only know EMH_combined exists
emh_combined = kaggle_change[~(kaggle_change['EMH_combined'].isna())]
# Then check their equality
both_values.equals(emh_combined)

True

We will make the following changes
1. __EMH and EMH_combined can be put into a single column, EMH, that has values {E, M, H, EM, MH, EMH}__
2. __In addition to the standard column changes, we will be making changes to other column names using the following map__

In [20]:
change_map = {'rate_at.5_chng_ach': 'achievement_dir',
              'rate_at.5_chng_growth': 'growth_dir',
              'pct_pts_chng_.5': 'overall_dir'}

3. __Finally, using the grading logic, we will map the direction columns using__

In [17]:
trend_arrow_map = {1: 'down', 
                  2: 'flat',
                  3: 'up'}

#### 2.3.2 COACT <a id="make-coact"></a>

In [11]:
kaggle_coact = pd.read_csv('../data/raw/kaggle/2010_COACT.csv')
kaggle_coact

,District No,2010 School Name,School No,eng_yn,math_yn,read_yn,sci_yn
0,10.0,MAPLETON EXPEDITIONARY SCHOOL OF THE ARTS,187.0,0.0,0.0,0.0,0.0
1,10.0,MAPLETON EARLY COLLEGE HIGH SCHOOL,212.0,1.0,0.0,0.0,0.0
2,10.0,WELBY NEW TECHNOLOGY,221.0,0.0,0.0,0.0,0.0
3,10.0,GLOBAL LEADERSHIP ACADEMY,263.0,0.0,0.0,0.0,0.0
4,10.0,SKYVIEW ACADEMY HIGH SCHOOL,309.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
kaggle_coact['eng_yn'].value_counts()

1.0    208
0.0    125
Name: eng_yn, dtype: int64

In [13]:
coact_2011 = pd.read_csv('../data/raw/kaggle/2011_COACT.csv')
coact_2011['eng_yn'].value_counts()

1.0    179
2.0    154
Name: eng_yn, dtype: int64

There is an inconsistency with the values. It seems that 2010 uses a different map than 2011 and 2012. We will assume that 1 means yes and 2 or 0 mean no

we will have to use slightly different column names to address the inconsistency with the other datasets

In [15]:
coact_2012 = pd.read_csv('../data/raw/kaggle/2012_COACT.csv')
coact_2012

,District No,School Name,District Name,School No,eng_yn,math_yn,read_yn,sci_yn
0,10,ACADEMY HIGH SCHOOL,MAPLETON 1,309,2.0,2.0,2.0,2.0
1,10,COLORADO CONNECTIONS ACADEMY,MAPLETON 1,1796,1.0,2.0,1.0,2.0
2,10,DISTRICT RESULTS,MAPLETON 1,0,2.0,2.0,2.0,2.0
3,10,GLOBAL LEADERSHIP ACADEMY,MAPLETON 1,263,2.0,2.0,2.0,2.0
4,10,MAPLETON EARLY COLLEGE HIGH SCHOOL,MAPLETON 1,212,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...
610,9030,YAMPAH TEEN PARENT PROGRAM,MOUNTAIN BOCES,9797,NaN,NaN,NaN,NaN
611,9050,DISTRICT RESULTS,SAN JUAN BOCES,0,NaN,NaN,NaN,NaN
612,9050,SOUTHWEST COLORADO E-SCHOOL,SAN JUAN BOCES,8121,NaN,NaN,NaN,NaN
613,9130,DISTRICT RESULTS,EXPEDITIONARY BOCES,0,1.0,2.0,2.0,2.0


We will also need to remove the rows that have district results as the school name.

#### 2.3.3 Enroll Working <a id='make-enrl'></a>

In [25]:
enrl_working = pd.read_csv('../data/raw/kaggle/2010_enrl_working.csv')
enrl_working

,Org. Code,Organization Name,School Code,School Name,TOTAL,PCT_AmInd,PCT_Asian,PCT_Black,PCT_hisp,PCT_White,PCT_PI,PCT_2ormore
0,10.0,MAPLETON 1,187.0,MAPLETON EXPEDITIONARY SCHOOL OF THE ARTS,543.0,0.012891,0.005525,0.033149,0.572744,0.344383,0.001842,0.029466
1,10.0,MAPLETON 1,212.0,FRONT RANGE EARLY COLLEGE,174.0,0.034483,0.005747,0.028736,0.568966,0.350575,0.000000,0.011494
2,10.0,MAPLETON 1,263.0,GLOBAL LEADERSHIP ACADEMY,447.0,0.002237,0.000000,0.026846,0.850112,0.109620,0.000000,0.011186
3,10.0,MAPLETON 1,309.0,SKYVIEW ACADEMY HIGH SCHOOL,311.0,0.003215,0.032154,0.012862,0.710611,0.234727,0.000000,0.006431
4,10.0,MAPLETON 1,501.0,MONTEREY COMMUNITY SCHOOL,385.0,0.012987,0.010390,0.005195,0.797403,0.161039,0.000000,0.012987
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,9030.0,MOUNTAIN BOCES,6134.0,YAMPAH MOUNTAIN SCHOOL,90.0,0.011111,0.000000,0.000000,0.366667,0.622222,0.000000,0.000000
1795,9030.0,MOUNTAIN BOCES,6136.0,MOUNTAIN BOCES DAY TREATMENT CENTER,26.0,0.000000,0.000000,0.000000,0.230769,0.769231,0.000000,0.000000
1796,9030.0,MOUNTAIN BOCES,9797.0,YAMPAH TEEN PARENT PROGRAM,26.0,0.000000,0.000000,0.000000,0.923077,0.076923,0.000000,0.000000
1797,9130.0,EXPEDITIONARY BOCES,2840.0,EXPEDITIONARY LEARNING SCHOOL,361.0,0.005540,0.038781,0.041551,0.102493,0.756233,0.000000,0.055402


Similar to COACT dataset we will only need to employ the standard changes but with slightly different column names to map from

#### 2.3.4 Final Grade

In [26]:
final = pd.read_csv('../data/raw/kaggle/2010_final_grade.csv')
final

,SchoolName,DistrictNumber,DistrictName,EMH,EMH_combined,SchoolNumber,AEC_10,CharterorOnline,LowestGrade,HighestGrade,...,Overall_ACH_Grade,Read_Ach_Grade,Math_Ach_Grade,Write_Ach_Grade,Sci_Ach_Grade,Overall_Weighted_Growth_Grade,Read_Growth_Grade,Math_Growth_Grade,Write_Growth_Grade,SPF_PS_IND_GRAD_RATE
0,BEAR CREEK ELEMENTARY SCHOOL,480,BOULDER VALLEY RE 2,E,NaN,652,0,NaN,Kindergarten,5th Grade,...,13.0,13.0,13.0,12.0,13.0,12.0,10.0,12.0,10.0,NaN
1,CHALLENGE SCHOOL,130,CHERRY CREEK 5,E,NaN,1510,0,NaN,Pre-Kindergarten,8th Grade,...,13.0,13.0,13.0,13.0,13.0,10.0,9.0,9.0,10.0,NaN
2,CHERRY HILLS VILLAGE ELEMENTARY SCHOOL,130,CHERRY CREEK 5,E,NaN,1574,0,NaN,Pre-Kindergarten,5th Grade,...,12.0,12.0,12.0,12.0,12.0,13.0,13.0,12.0,13.0,NaN
3,DRY CREEK ELEMENTARY SCHOOL,130,CHERRY CREEK 5,E,NaN,2292,0,NaN,Pre-Kindergarten,5th Grade,...,12.0,12.0,12.0,12.0,10.0,13.0,12.0,10.0,13.0,NaN
4,STECK ELEMENTARY SCHOOL,880,DENVER COUNTY 1,E,NaN,8222,0,NaN,Pre-Kindergarten,5th Grade,...,13.0,13.0,13.0,13.0,12.0,13.0,13.0,12.0,13.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,IVER C. RANUM MIDDLE SCHOOL,70,WESTMINSTER 50,NaN,NaN,7305,0,NaN,6th Grade,8th Grade,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2078,DISCOVERY HIGH SCHOOL,990,WIDEFIELD 3,H,NaN,9560,0,NaN,9th Grade,12th Grade,...,5.0,5.0,3.0,4.0,5.0,1.0,3.0,2.0,1.0,29.2
2079,RANGE VIEW ELEMENTARY,3100,WINDSOR RE-4,NaN,NaN,8459,0,NaN,Pre-Kindergarten,5th Grade,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2080,TOZER ELEMENTARY SCHOOL,3100,WINDSOR RE-4,NaN,NaN,8886,0,NaN,Pre-Kindergarten,2nd Grade,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
final.columns

Index(['SchoolName', 'DistrictNumber', 'DistrictName', 'EMH', 'EMH_combined',
       'SchoolNumber', 'AEC_10', 'CharterorOnline', 'LowestGrade',
       'HighestGrade', 'INITIAL_PlanType', 'FINAL_PlanType', 'Notes',
       'EMH_2lvl', 'LT100pnts', 'School_Grade', 'rank_tot',
       'Overall_ACH_Grade', 'Read_Ach_Grade', 'Math_Ach_Grade',
       'Write_Ach_Grade', 'Sci_Ach_Grade', 'Overall_Weighted_Growth_Grade',
       'Read_Growth_Grade', 'Math_Growth_Grade', 'Write_Growth_Grade',
       'SPF_PS_IND_GRAD_RATE'],
      dtype='object')

In [33]:
# Check the content of Notes
final['Notes']

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
2077                     New school, deferred to district
2078                       Approved request to reconsider
2079                                           New school
2080    Insufficient state evidence, deferred to district
2081                                                  NaN
Name: Notes, Length: 2082, dtype: object

In [34]:
# Determine number of missing values to decided whether or not to keep it
final['Notes'].isna().sum()

1690

We will keep the column for now. It might be interesting to investigate during in EDA when notes are taken. For instance, perhaps they are only being recorded when schools perform well or poorly

In [37]:
# There is no description offered that explains this feature. Let's check out it's values to see if we can make sense of it
final['EMH_2lvl'].value_counts()

1.0    1576
2.0     446
Name: EMH_2lvl, dtype: int64

It remains mysterious, so we will drop it

In [42]:
# Let's see how these features are indicated
final['INITIAL_PlanType'].value_counts()

Performance Plan             1323
Improvement Plan              383
Priority Improvement Plan     172
Alternative Ed. Campus         87
Turnaround Plan                82
Name: INITIAL_PlanType, dtype: int64

It seems wordy, perhaps we can reduce the words

In [43]:
final['FINAL_PlanType'].value_counts()

Performance Plan             1318
Improvement Plan              471
Priority Improvement Plan     163
Turnaround Plan                82
School Closed                  48
Name: FINAL_PlanType, dtype: int64

In [50]:
final['AEC_10'].value_counts()

0    1989
1      93
Name: AEC_10, dtype: int64

There might be some issues here. We can see that the Final plan does not contain the Alternative Ed. Campus as an option, and without counting, it seems like these might have more missing values. Maybe alternative education isn't relevant to this column and maybe it needs to be transposed from the initial plan type

In [44]:
final['CharterorOnline'].value_counts()

Charter             273
Online               32
Charter & Online      8
Name: CharterorOnline, dtype: int64

We might want to do a similar thing as the EMH column here when we need to do feature extraction for creating a model. For now, it can stay

In [45]:
# There is no description offered that explains this feature. Let's check out it's values to see if we can make sense of it
final['LT100pnts'].value_counts()

0    1918
1     164
Name: LT100pnts, dtype: int64

It still doesn't make sense, so we will drop it

In [47]:
# Like in the change dataset, 
# let's test to see if the EMH and EMH_combined columns can be combined
# Create dataframes for where we know EMH and EMH_combined exists
both_values = final[~(final['EMH'].isna()) & ~(final['EMH_combined'].isna())]
# And where we only know EMH_combined exists
emh_combined = final[~(final['EMH_combined'].isna())]
# Then check their equality
both_values.equals(emh_combined)

True

And they can be

##### Summary of all changes to be made to Final Grade
1. We will drop the following columns

In [46]:
final_drop = ['EMH_2lvl', 'LT100pnts']

2. We will combine the EMH and EMH_combined columns as described in 1YR_3YR_change datasets
3. We will change the column names according to the following map

In [53]:
final_map = {'AEC_10': 'alternative_school',
             'INITIAL_PlanType': 'initial_plan',
             'FINAL_PlanType': 'final_plan', 
             'rank_tot': 'rank', 
             'Overall_ACH_Grade': 'overall_achievement',
             'Read_Ach_Grade': 'read_achievement',
             'Math_Ach_Grade': 'math_achievement',
             'Write_Ach_Grade': 'write_achievement',
             'Sci_Ach_Grade': 'science_achievment',
             'Overall_Weighted_Growth_Grade': 'overall_weighted_growth',
             'Read_Growth_Grade': 'read_growth',
             'Math_Growth_Grade': 'math_growth',
             'Write_Growth_Grade': 'write_growth',
             'SPF_PS_IND_GRAD_RATE': 'graduation_rate'}

#### 2.3.5 Free or Reduced Lunch <a id='make-frl'></a>

In [14]:
frl = pd.read_csv('../data/raw/kaggle/2010_k_12_FRL.csv')
frl

,DISTRICT CODE,DISTRICT NAME,SCHOOL CODE,SCHOOL NAME,% FREE AND REDUCED
0,10.0,MAPLETON 1,187.0,MAPLETON EXPEDITIONARY SCHOOL OF THE ARTS,65.6%
1,10.0,MAPLETON 1,212.0,MAPLETON EARLY COLLEGE HIGH SCHOOL,69.0%
2,10.0,MAPLETON 1,263.0,GLOBAL LEADERSHIP ACADEMY,90.6%
3,10.0,MAPLETON 1,309.0,SKYVIEW ACADEMY HIGH SCHOOL,73.3%
4,10.0,MAPLETON 1,501.0,MONTEREY COMMUNITY SCHOOL,80.8%
...,...,...,...,...,...
1750,9030.0,MOUNTAIN BOCES,NaN,BOCES TOTALS,0.0%
1751,9130.0,EXPEDITIONARY BOCES,2840.0,EXPEDITIONARY LEARNING SCHOOL,0.0%
1752,9130.0,EXPEDITIONARY BOCES,NaN,BOCES TOTALS,0.0%
1753,NaN,NaN,NaN,COLORADO DETENTION CENTERS,23.8%


Other than the standard changes we will change the column the % FREE AND REDUCED to a more readable format change its column type to a float. We will also drop the last to rows, since they are not what is needed for the project

In [13]:
FRL_COL_MAP = {'% FREE AND REDUCED': 'pct_fr'}

In [14]:
frl = frl.rename(columns=FRL_COL_MAP)

In [15]:
frl['pct_fr'] = frl['pct_fr'].str.replace('%','').astype('float') / 100

In [16]:
frl.head()

,DISTRICT CODE,DISTRICT NAME,SCHOOL CODE,SCHOOL NAME,pct_fr
0,10.0,MAPLETON 1,187.0,MAPLETON EXPEDITIONARY SCHOOL OF THE ARTS,0.656
1,10.0,MAPLETON 1,212.0,MAPLETON EARLY COLLEGE HIGH SCHOOL,0.690
2,10.0,MAPLETON 1,263.0,GLOBAL LEADERSHIP ACADEMY,0.906
3,10.0,MAPLETON 1,309.0,SKYVIEW ACADEMY HIGH SCHOOL,0.733
4,10.0,MAPLETON 1,501.0,MONTEREY COMMUNITY SCHOOL,0.808


#### 2.3.6 Remediation <a id='make-remediation'></a>

In [17]:
rem = pd.read_csv('../data/raw/kaggle/2010_remediation_HS.csv')
rem.head()

,School_District,SchoolName,SchoolNumber,Remediation_AtLeastOne_Pct2010
0,DENVER COUNTY 1,ABRAHAM LINCOLN HIGH SCHOOL,10,0.785714
1,NORTHGLENN-THORNTON 12,ACADEMY OF CHARTER SCHOOLS,15,NaN
2,ADAMS COUNTY 14,ADAMS CITY HIGH SCHOOL,24,0.569231
3,JEFFERSON COUNTY R-1,MC LAIN HIGH SCHOOL,33,0.545455
4,ACADEMY 20,AIR ACADEMY HIGH SCHOOL,76,0.147368


We will change the remediation percent column to something more readable. That is all that needs to be done

In [18]:
REM_COL_MAP = {'Remediation_AtLeastOne_Pct2010': 'pct_remediation'}

#### 2.3.7 Addresses <a id='make-address'></a>

In [19]:
address = pd.read_csv('../data/raw/kaggle/2010_school_address.csv')
address.head()

,Organization Code,School Code,Phone,School Name,Physical Address,Physical City,Physical State,Physical Zipcode
0,880,10,7204235000,Abraham Lincoln High School,2285 South Federal Boulevard,Denver,CO,80219
1,900,11,3036604881,Academy Charter School,1551 Prairie Hawk Drive,Castle Rock,CO,80104
2,900,12,3033877125,Acres Green Elementary School,13524 North Acres Green Drive,Littleton,CO,80124
3,20,14,7209725940,Glacier Peak Elementary School,12060 Jasmine Street,Brighton,CO,80605
4,20,15,3032898088,Academy Of Charter Schools,11800 Lowell Blvd,Westminster,CO,80031


The phone number column is irrelevant to our project, so we will drop it. We will also drop the address column because we do not need that level of specificity. Everything else can be changed as expected

In [21]:
ADDRESS_COL_DROP = ['Phone', 'Physical Address']

In [22]:
ADDRESS_COL_MAP = {'Physical City': 'city',
                  'Physical State': 'state',
                  'Physical Zipcode': 'zipcode'}

## Conclusion
Data Wrangling was extremely expensive for this project. 
- The Expenditures data was in a report style format that took extensive manipulation to convert it to a useful format.
- The Kaggle Data was filled with inconsistencies, poor data explanations, and useless information.
Overall this project took days to complete and even after doing it there are still errors and issues to deal with in the Exploratory Data Analysis and the Preprocessing phases.



*Note: Please refer to src/data/make_datasets.py for more details on how the exploration in this notebook was applied and debugged*